In [ ]:
# !pip install qiskit

# VQLS with Qiskit
**Course**: Applied Quantum Algorithms

**Date**: April 20, 2022

In this notebook, we will solve a linear system of the form
$$
A|x\rangle = |b\rangle
$$
by implementing the VQLS algorithm in Qiskit.

We will first build some of the functions ourselves and verify them, but then use the compact libraries of Qiskit to build the final version of the cost function for optimization.

In [ ]:
import numpy as np
from qiskit import Aer
from qiskit.circuit.library.n_local import TwoLocal
from qiskit.opflow import (Z, I, H, CircuitStateFn, StateFn)
from qiskit.quantum_info import Statevector

## Ansatz

First off, we know that all hybrid variational algorithms use a parametrized quantum circuit $U(\theta)$ to represent the tentative solution as

$$
|\psi(\theta)\rangle = U(\theta) |0\rangle^{\otimes n},
$$

where $N$ is the dimension of our linear system and $n=\log_2 N$.

For this exercise, we want to build the following ansatz on 3 qubits

![ansatz](ansatz.png)

Notice that the CZ-gates are arranged with a *full-entanglement* logic, so controls and targets are respectively:
1. 0 -> 1
2. 0 -> 2
3. 1 -> 2

In [ ]:
from qiskit import QuantumCircuit

def build_ansatz(params):
    """
    Returns the ansatz circuit for representing |\psi(\theta)>
    
    Arguments:
        params: np.ndarray containing the parameters of the rotation gates
    """
    qc = QuantumCircuit(3)
    
    # ===============
    # YOUR CODE BELOW
    # ===============
    
    return qc

Let's check our implementation using some random parameters as argument to our function

In [ ]:
rng = np.random.default_rng(1)

theta_0 = rng.uniform(-2 * np.pi, 2 * np.pi, size=6)

ansatz = build_ansatz(theta_0)

ansatz.draw(output='mpl')

Instead of creating parametrized circuits yourself, Qiskit offers a set of ansatz templates that you can directly use. For instance, we can use the `TwoLocal` class to produce the same ansatz that we just created

In [ ]:
ansatz = TwoLocal(3, 'ry', 'cz', 'full', reps=1, insert_barriers=True)
qc = ansatz.assign_parameters(theta_0).decompose()
qc.draw(output='mpl')

## Expectation value

Let's assume that our problem is the following:

Find $|x\rangle$, such that $A|x\rangle=|b\rangle$, where

$$
A = 0.55 I + 0.225 Z_1 + 0.225 Z_2
$$

and

$$
|b\rangle = B |0\rangle^{\otimes 3} = H^{\otimes 3} |0\rangle^{\otimes 3}.
$$

We know that a possible cost function used in VQLS is

$$
C(\theta) = \langle\psi(\theta)| A^\dagger (I - |b\rangle\langle b|) A |\psi(\theta)\rangle,
$$

which we can also write as

$$
C(\theta) = \langle\psi(\theta)| A^\dagger A |\psi(\theta)\rangle - \langle\psi(\theta)| A^\dagger B |0\rangle\langle 0 | B^\dagger A |\psi(\theta)\rangle.
$$

Thus, we need to evaluate terms such 

In [ ]:
backend = Aer.get_backend('statevector_simulator')

In [ ]:
A = 0.55 * (I ^ I ^ I) \
    + 0.225 * (I ^ Z ^ I) \
        + 0.225 * (I ^ I ^ Z)

B = (H ^ H ^ H).to_pauli_op()

In [ ]:
A

In [ ]:
B

In [ ]:
A_squared = (~A) @ A

In [ ]:
A_squared

In [ ]:
zero_proj = 0.5 * (I + Z)
zero_proj_3 = zero_proj ^ zero_proj ^ zero_proj

In [ ]:
print(zero_proj_3.to_matrix())

In [ ]:
A_b_proj = (~A) @ (~B) @ zero_proj_3 @ B @ A

In [ ]:
print(A_b_proj)

In [ ]:
ansatz = TwoLocal(3, 'ry', 'cz', 'full', reps=1, insert_barriers=True)
# fig = ansatz.decompose().draw(output='mpl')
# fig.savefig('ansatz.png', bbox_inches='tight')
print(ansatz.assign_parameters(theta_0).decompose().qasm())

In [ ]:
theta_0 = np.random.uniform(-2 * np.pi, 2 * np.pi, size=ansatz.num_parameters)
# print(theta_0)
# psi = CircuitStateFn(ansatz.assign_parameters(theta_0))
# print(psi)

In [ ]:
ev1 = StateFn(A_squared).adjoint().eval(psi)
ev2 = StateFn(A_b_proj).adjoint().eval(psi)
print(ev1.real)
print(ev2.real)
# expectation = AerPauliExpectation().convert(measurable)
# sampler = CircuitSampler(backend).convert(expectation)  
# print('Snapshot:', sampler.eval().real) 

In [ ]:
def cost(params):
    """
    Returns the global un-normalized cost function for VQLS:
    C_G = <\psi| (~A (1-|b><b|) A) |\psi>
    """
    psi = CircuitStateFn(ansatz.assign_parameters(params))
    
    ev1 = StateFn(A_squared).adjoint().eval(psi)
    ev2 = StateFn(A_b_proj).adjoint().eval(psi)

    return ev1.real - ev2.real

In [ ]:
out = minimize(cost, x0=theta_0, method="COBYLA", options={'maxiter':500})
print(out)

In [ ]:
theta_opt = out.x
zero_state = Statevector.from_label('000')
psi_opt = zero_state.evolve(ansatz.assign_parameters(theta_opt))
psi_opt_vector = psi_opt.data
print(psi_opt_vector)

In [ ]:
A.to_matrix()
# B.to_matrix()

In [ ]:
np.linalg.norm((A.to_matrix().real @ psi_opt_vector.real) - (1/np.sqrt(8) * np.ones(8)))

In [ ]:
zero_state = Statevector.from_label('000')
b_ket = B.to_matrix() @ zero_state.data
print(b_ket)
print(1/np.sqrt(8) * np.ones(8))

In [ ]:
x_true = np.linalg.solve(A.to_matrix(), (1/np.sqrt(8) * np.ones(8)))
print(x_true)
print(np.linalg.norm(x_true))

In [ ]:
x_true[0] / psi_opt_vector[0]

In [ ]:
5.1822 * psi_opt_vector